In [1]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="tiendoan/correctDoancs221",device = 'cuda')

2024-07-02 03:16:21.378340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 03:16:21.378445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 03:16:21.516781: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/961 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset
test_dataset = load_dataset("tiendoan/ngrams_vnc_dataset",split = 'test')

Generating train split:   0%|          | 0/6579290 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/35999 [00:00<?, ? examples/s]

In [3]:
import re
def extract_phrases(text):
        pattern = r'\w[\w ]*|\s\W+|\W+'

        phrases_all = re.findall(pattern, text)

        index_sent_dict = {}
        phrases_str = []
        for ind, phrase in enumerate(phrases_all):
            if not re.match(r'[!"#$%&''()*+,-./:;<=>?@[\]^_`{|}~]', phrase.strip()):
                phrases_str.append(phrase.strip())
                index_sent_dict[ind] = phrase

        return phrases_str, phrases_all, index_sent_dict


In [4]:
from tqdm import tqdm
extracted_sens = []
for sen in tqdm(test_dataset['input']):
    extracted_sens.append(extract_phrases(sen)[1])

100%|██████████| 35999/35999 [00:00<00:00, 37138.14it/s]


In [5]:
def predict(text,ngram = 5):
    words = text.split()
    sen = []
    pred = []
    i = 1
    for word in (words):
        if(i % ngram == 0):
            pr = corrector(' '.join(sen),max_length=128)[0]['generated_text']
            pred.append(pr)
            sen = []
        sen.append(word)
        i = i + 1
    pred.append(corrector(' '.join(sen))[0]['generated_text'])
    return ' '.join(pred)

In [6]:
preds = []
for sen in tqdm(extracted_sens): 
    pred = []
    for i in range(len(sen)):
        if(i%2 == 0):  
            pred.append(predict(sen[i],ngram = 5))
        else: pred.append(sen[i])
    preds.append(''.join(pred))

  0%|          | 0/35999 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  4%|▎         | 1319/35999 [13:43<6:01:00,  1.60it/s] 

KeyboardInterrupt



In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics.functional.text import char_error_rate, word_error_rate

In [ ]:
labels = test_dataset['output']

In [ ]:
char_error_rate(preds=preds, target=labels)

In [ ]:
word_error_rate(preds=preds, target=labels)